In [0]:
%sql
-- Create the source table with some initial data
CREATE TABLE source_table (
    id INT,
    name STRING,
    last_modified TIMESTAMP
);

-- Insert initial data into the source table
INSERT INTO source_table VALUES
(1, 'Alice', '2024-08-25 10:00:00'),
(2, 'Bob', '2024-08-25 10:05:00'),
(3, 'Charlie', '2024-08-25 10:10:00');

num_affected_rows,num_inserted_rows
3,3


In [0]:
%sql
select * from source_table

id,name,last_modified
1,Alice,2024-08-25T10:00:00.000+0000
2,Bob,2024-08-25T10:05:00.000+0000
3,Charlie,2024-08-25T10:10:00.000+0000


In [0]:
%sql
-- Create the target table by copying all data from the source table
CREATE TABLE target_table AS
SELECT * FROM source_table;


num_affected_rows,num_inserted_rows


In [0]:
%sql
-- Insert a new record into the source table
INSERT INTO source_table VALUES
(4, 'David', '2024-08-25 10:15:00');

-- Update an existing record in the source table
UPDATE source_table SET 
    name = 'Alice Updated',
    last_modified = '2024-08-25 10:20:00'
WHERE id = 1;


num_affected_rows
1


In [0]:
%sql
select * from source_table

id,name,last_modified
1,Alice Updated,2024-08-25T10:20:00.000+0000
2,Bob,2024-08-25T10:05:00.000+0000
3,Charlie,2024-08-25T10:10:00.000+0000
4,David,2024-08-25T10:15:00.000+0000


In [0]:
%sql
-- Identify incremental data based on the last_modified column
CREATE OR REPLACE TEMP VIEW incremental_data AS
SELECT *
FROM source_table
WHERE last_modified > (SELECT MAX(last_modified) FROM target_table);

In [0]:
%sql
-- Perform the merge operation to update and insert records
MERGE INTO target_table AS tgt
USING incremental_data AS src
ON tgt.id = src.id
WHEN MATCHED THEN
    UPDATE SET tgt.name = src.name, tgt.last_modified = src.last_modified
WHEN NOT MATCHED THEN
    INSERT (id, name, last_modified) VALUES (src.id, src.name, src.last_modified);

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
2,1,0,1


In [0]:
%sql
-- Verify the result in the target table
SELECT * FROM target_table

id,name,last_modified
1,Alice Updated,2024-08-25T10:20:00.000+0000
4,David,2024-08-25T10:15:00.000+0000
2,Bob,2024-08-25T10:05:00.000+0000
3,Charlie,2024-08-25T10:10:00.000+0000


In [0]:
%sql
-- Create the Source table where new data is added or updated
CREATE TABLE SourceEmployees (
    EmployeeID INT,
    FirstName STRING,
    LastName STRING,
    Department STRING,
    Salary DECIMAL(10, 2),
    LastModifiedDate DATE
)
USING DELTA;

-- Create the Target table where data will be merged
CREATE TABLE TargetEmployees (
    EmployeeID INT,
    FirstName STRING,
    LastName STRING,
    Department STRING,
    Salary DECIMAL(10, 2),
    LastModifiedDate DATE
)
USING DELTA;


In [0]:
%sql
-- Insert sample data into SourceEmployees
INSERT INTO SourceEmployees VALUES
(1, 'John', 'Doe', 'Engineering', 75000.00, '2024-08-10'),
(2, 'Jane', 'Smith', 'Marketing', 68000.00, '2024-08-11'),
(3, 'Alice', 'Johnson', 'HR', 62000.00, '2024-08-12');


num_affected_rows,num_inserted_rows
3,3


In [0]:
%sql
select * from SourceEmployees

EmployeeID,FirstName,LastName,Department,Salary,LastModifiedDate
1,John,Doe,Engineering,75000.00,2024-08-10
2,Jane,Smith,Marketing,68000.00,2024-08-11
3,Alice,Johnson,HR,62000.00,2024-08-12


In [0]:
%sql
-- Insert initial data into TargetEmployees
INSERT INTO TargetEmployees VALUES
(1, 'John', 'Doe', 'Engineering', 70000.00, '2024-08-01'),  -- Older salary and LastModifiedDate
(2, 'Jane', 'Smith', 'Marketing', 68000.00, '2024-08-11');  -- No change, same as in SourceEmployees


num_affected_rows,num_inserted_rows
2,2


In [0]:
%sql
-- Merge SourceEmployees into TargetEmployees with incremental logic
MERGE INTO TargetEmployees AS target
USING SourceEmployees AS source
ON target.EmployeeID = source.EmployeeID

WHEN MATCHED AND target.LastModifiedDate < source.LastModifiedDate THEN
  UPDATE SET 
    target.FirstName = source.FirstName,
    target.LastName = source.LastName,
    target.Department = source.Department,
    target.Salary = source.Salary,
    target.LastModifiedDate = source.LastModifiedDate

WHEN NOT MATCHED THEN
  INSERT (EmployeeID, FirstName, LastName, Department, Salary, LastModifiedDate)
  VALUES (source.EmployeeID, source.FirstName, source.LastName, source.Department, source.Salary, source.LastModifiedDate);


num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
2,1,0,1


In [0]:
%sql
select * from TargetEmployees

EmployeeID,FirstName,LastName,Department,Salary,LastModifiedDate
2,Jane,Smith,Marketing,68000.00,2024-08-11
1,John,Doe,Engineering,75000.00,2024-08-10
3,Alice,Johnson,HR,62000.00,2024-08-12


In [0]:
%sql
-- Step 1: Create the Source table where new data is added or updated
CREATE TABLE SourceEmployeeRecords (
    EmployeeID INT,
    FirstName STRING,
    LastName STRING,
    Department STRING,
    Salary DECIMAL(10, 2),
    LastModifiedDate DATE
)
USING DELTA;

-- Step 2: Create the Target table where data will be merged
CREATE TABLE TargetEmployeeRecords (
    EmployeeID INT,
    FirstName STRING,
    LastName STRING,
    Department STRING,
    Salary DECIMAL(10, 2),
    LastModifiedDate DATE
)
USING DELTA;

-- Step 3: Insert initial data into SourceEmployeeRecords
INSERT INTO SourceEmployeeRecords 
VALUES 
    (1, 'John', 'Doe', 'Engineering', 75000.00, '2024-08-10'),
    (2, 'Jane', 'Smith', 'Marketing', 68000.00, '2024-08-11'),
    (3, 'Alice', 'Johnson', 'HR', 62000.00, '2024-08-12');

-- Step 4: Insert initial data into TargetEmployeeRecords
INSERT INTO TargetEmployeeRecords 
VALUES 
    (1, 'John', 'Doe', 'Engineering', 70000.00, '2024-08-01'),  -- Older salary and LastModifiedDate
    (2, 'Jane', 'Smith', 'Marketing', 68000.00, '2024-08-11');  -- No change, same as in SourceEmployeeRecords

-- Step 5: Insert additional data into SourceEmployeeRecords
INSERT INTO SourceEmployeeRecords 
VALUES 
    (2, 'Jane', 'Smith', 'Sales', 70000.00, '2024-08-15'),  -- Updated record (Department changed, Salary increased)
    (4, 'Bob', 'Brown', 'Engineering', 80000.00, '2024-08-16');  -- New record

-- Step 6: Apply incremental load using window function and MERGE INTO
WITH RankedSource AS (
  SELECT *, 
         ROW_NUMBER() OVER (PARTITION BY EmployeeID ORDER BY LastModifiedDate DESC) AS rn
  FROM SourceEmployeeRecords
  WHERE EmployeeID IS NOT NULL -- Ensure no nulls are considered
)

MERGE INTO TargetEmployeeRecords AS target
USING (SELECT * FROM RankedSource WHERE rn = 1) AS source -- Only take the latest row per EmployeeID
ON target.EmployeeID = source.EmployeeID

WHEN MATCHED AND target.LastModifiedDate < source.LastModifiedDate THEN
  UPDATE SET target.FirstName = source.FirstName,
             target.LastName = source.LastName,
             target.Department = source.Department,
             target.Salary = source.Salary,
             target.LastModifiedDate = source.LastModifiedDate

WHEN NOT MATCHED THEN
  INSERT (EmployeeID, FirstName, LastName, Department, Salary, LastModifiedDate)
  VALUES (source.EmployeeID, source.FirstName, source.LastName, source.Department, source.Salary, source.LastModifiedDate);

-- Step 7: Verify the results in TargetEmployeeRecords
SELECT * FROM TargetEmployeeRecords;


EmployeeID,FirstName,LastName,Department,Salary,LastModifiedDate
1,John,Doe,Engineering,75000.00,2024-08-10
2,Jane,Smith,Sales,70000.00,2024-08-15
3,Alice,Johnson,HR,62000.00,2024-08-12
4,Bob,Brown,Engineering,80000.00,2024-08-16


In [0]:
%sql
-- Drop the SourceEmployeeRecords table
DROP TABLE IF EXISTS SourceEmployeeRecords;

-- Drop the TargetEmployeeRecords table
DROP TABLE IF EXISTS TargetEmployeeRecords;
